In [1]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob

### Import Output datas

In [2]:

sp_iso_path='/Users/pablogarcia44/repo/Garcia_OpenSn/pincell_study/1_legendre/order0_openmc/sp_iso.h5'
sm_iso_path='/Users/pablogarcia44/repo/Garcia_OpenSn/pincell_study/1_legendre/order0_openmc/sm_iso.h5'
model_iso_path='/Users/pablogarcia44/repo/Garcia_OpenSn/pincell_study/1_legendre/order0_openmc/model_iso.xml'

In [5]:
sm=openmc.Summary(sm_iso_path)
sp = openmc.StatePoint(sp_iso_path)
sp.link_with_summary(sm)
model=openmc.model.Model
model=openmc.model.Model.from_model_xml(model_iso_path)
mgxs_lib = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(openmc.mgxs.GROUP_STRUCTURES['XMAS-172'])
mgxs_lib.energy_groups = groups
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 0
mgxs_lib.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,'chi', 'consistent scatter matrix', 'multiplicity matrix','total','consistent nu-scatter matrix']
mgxs_lib.by_nuclide = False
mgxs_lib.domain_type = 'cell'
mgxs_lib.domains = model.geometry.get_all_material_cells().values()
mgxs_lib.build_library()
mgxs_lib.load_from_statepoint(sp)


/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=7.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=8.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=9.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=7.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=8.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=9.
  warn(msg, IDWarning)
/opt/an

In [13]:
mgxs_lib_order1=load_mgxs(sp_order1_path,sm_order1_path,model_order1_path)
mgxs_file_order1 = mgxs_lib_order1.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order1.xsdatas[0].temperatures=[565.]
mgxs_file_order1.xsdatas[1].temperatures=[565.]
mgxs_file_order1.xsdatas[2].temperatures=[565.]


/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=4.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=5.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=6.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=7.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=8.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=9.
  warn(msg, IDWarning)
/opt/an

In [ ]:
model=openmc.Model()
mgxs_file_order1, materials_file, geometry_file = mgxs_lib.create_mg_mode()

iso_mgxs_file.xsdatas[0].temperatures=[565.]
iso_mgxs_file.xsdatas[1].temperatures=[565.]
iso_mgxs_file.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_iso.h5'

iso_mgxs_file.export_to_hdf5('mgxs_iso.h5')
model2.materials = materials_file
model2.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model2.settings = settings_file

### Post Processing

In [45]:
def cell_domain(cell,mgxs_lib):
    for k in range(len(mgxs_lib.domains)):
        if mgxs_lib.domains[k].name==cell:
            return(mgxs_lib.domains[k].id)

In [46]:
def mgxs_value(cell,reaction,mgxs_lib):  
    cell_name=cell
    cell=cell_domain(cell_name,mgxs_lib)
    mgxs_plot = mgxs_lib.get_mgxs(cell,reaction)
    mgxs=mgxs_plot.get_pandas_dataframe(xs_type='macro')['mean'].values
    return(mgxs)

In [66]:
def rd_max(mgxs1,mgxs2):   
    avg=0
    index=0
    rd_max=0
    for i in range (171):
        rd1=0
        rd2=0
        if mgxs1[i]!=0:
            rd1=abs(mgxs1[i]-mgxs2[i])/mgxs1[i]
        if mgxs2[i]!=0:
            rd2=abs(mgxs1[i]-mgxs2[i])/mgxs2[i]
        if rd1>rd_max or rd1>rd_max:
            rd_max=max(rd1,rd2)
            index=i
    return(100*rd_max,index)      

In [69]:
fuel_1=mgxs_value('fuel_1_family','fission',mgxs_lib_1_family)
fuel_E=mgxs_value('fuel_E','fission',mgxs_lib_6_families)
fuel_C=mgxs_value('fuel_C','fission',mgxs_lib_6_families)
fuel_N=mgxs_value('fuel_N','fission',mgxs_lib_6_families)


print(rd_max(fuel_N,fuel_C))
print(rd_max(fuel_N,fuel_E))
print(rd_max(fuel_E,fuel_C))

(9.081669831081102, 79)
(3.55279605429432, 64)
(7.794569216902131, 79)
